In [ ]:
!pip install kagglehub
import kagglehub
import os

kaggle_path = kagglehub.dataset_download("humansintheloop/car-parts-and-car-damages")
print("Датасет Kaggle скачан в:", kaggle_path)
print("\nСодержимое папки Kaggle:")
!ls {kaggle_path}

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="RHcpuYZ2kGtOP3mq8eyT")
project = rf.workspace("dirty").project("less-than-640")
version = project.version(2)
roboflow_dataset = version.download("folder")


print("Датасет Roboflow успешно скачан в:", roboflow_dataset.location)

# Проверяем содержимое папки, чтобы убедиться, что все на месте
print("\nСодержимое папки Roboflow:")
!ls {roboflow_dataset.location}


Using Colab cache for faster access to the 'car-parts-and-car-damages' dataset.
Датасет Kaggle скачан в: /kaggle/input/car-parts-and-car-damages

Содержимое папки Kaggle:
'Car damages dataset'  'Car parts dataset'
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 999, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/build_env.py", line 21, in <module>
    from pip._internal.metadata import get_default_environment, get_environment
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/metadata/__init__.py", line 9, in <module>
    from .base import BaseDistribution, BaseEnvironment, FilesystemWheel, MemoryWheel, Wheel
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/metadata/base.

In [ ]:
# Устанавливаем PyTorch и другие библиотеки при необходимости (скриптовый способ, работает и в Colab)
import sys, subprocess
try:
    import torch  # noqa: F401
    import torchvision  # noqa: F401
    import torchaudio  # noqa: F401
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch", "torchvision", "torchaudio"])

import os
import math
import copy
import random
from typing import Dict, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, models, transforms
from torch.optim.lr_scheduler import CosineAnnealingLR

# Пути к данным, которые ты скачал с Roboflow
# Примечание: путь ниже рассчитан на Google Colab. Замени при необходимости.
data_dir = '/content/less-than-640-2'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'valid')

# Фиксируем сиды для воспроизводимости (по возможности)
def set_seed(seed: int = 42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = False  # True замедляет, но детерминирует
    torch.backends.cudnn.benchmark = True

set_seed(42)

# Определяем преобразования для изображений (минимальные без аугментаций)
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Создаем объекты датасетов
image_datasets = {
    'train': datasets.ImageFolder(train_dir, data_transforms['train']),
    'val': datasets.ImageFolder(val_dir, data_transforms['val'])
}

# Удаляем класс 'Unlabeled' (если присутствует) из наборов
def remove_class_from_imagefolder(ds: datasets.ImageFolder, class_name: str = 'Unlabeled') -> datasets.ImageFolder:
    if class_name not in ds.class_to_idx:
        return ds
    remove_idx = ds.class_to_idx[class_name]
    new_classes = [c for c in ds.classes if c != class_name]
    new_class_to_idx = {c: i for i, c in enumerate(new_classes)}

    new_samples = []
    new_targets = []
    removed = 0
    for path, old_y in ds.samples:
        if old_y == remove_idx:
            removed += 1
            continue
        class_name_for_old = ds.classes[old_y]
        new_y = new_class_to_idx[class_name_for_old]
        new_samples.append((path, new_y))
        new_targets.append(new_y)

    ds.samples = new_samples
    ds.imgs = new_samples
    ds.targets = new_targets
    ds.classes = new_classes
    ds.class_to_idx = new_class_to_idx
    print(f"Removed class '{class_name}': {removed} samples removed.")
    return ds

# Применяем фильтрацию
image_datasets['train'] = remove_class_from_imagefolder(image_datasets['train'], 'Unlabeled')
image_datasets['val'] = remove_class_from_imagefolder(image_datasets['val'], 'Unlabeled')

# Оценим дисбаланс классов и при необходимости используем WeightedRandomSampler
def build_train_sampler_if_needed(dataset: datasets.ImageFolder, imbalance_threshold: float = 1.5):
    # Считаем количество образцов по классам
    targets = [y for _, y in dataset.samples]
    class_counts = {}
    for y in targets:
        class_counts[y] = class_counts.get(y, 0) + 1
    max_c = max(class_counts.values())
    min_c = min(class_counts.values())
    ratio = max_c / max(1, min_c)
    if ratio >= imbalance_threshold:
        class_weights = {c: (1.0 / count) for c, count in class_counts.items()}
        sample_weights = [class_weights[y] for y in targets]
        sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
        return sampler, class_counts, ratio
    return None, class_counts, ratio

train_sampler, class_counts, imbalance_ratio = build_train_sampler_if_needed(image_datasets['train'])

# Создаем загрузчики данных
dataloaders = {
    'train': DataLoader(
        image_datasets['train'], batch_size=32,
        shuffle=(train_sampler is None), sampler=train_sampler,
        num_workers=4, pin_memory=True
    ),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
}

# Определяем размер датасетов (после фильтрации)
dataset_sizes = {
    'train': len(image_datasets['train']),
    'val': len(image_datasets['val'])
}

# Получаем имена классов (после фильтрации)
class_names = image_datasets['train'].classes
print("Имена классов:", class_names)
print("Размер обучающего набора:", dataset_sizes['train'])
print("Размер валидационного набора:", dataset_sizes['val'])
print("Распределение классов (train):", {class_names[c]: n for c, n in class_counts.items()})
if imbalance_ratio >= 1.5:
    print(f"Обнаружен дисбаланс классов ≈ {imbalance_ratio:.2f}×. Включен WeightedRandomSampler.")

# Проверка согласованности наборов классов между train и val
if set(image_datasets['train'].classes) != set(image_datasets['val'].classes):
    raise RuntimeError(
        f"Наборы классов в train и val отличаются. train={image_datasets['train'].classes}, val={image_datasets['val'].classes}.\n"
        "Убедись, что папки классов совпадают (например, только 'clean' и 'dirty')."
    )

# Проверяем, доступен ли GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Используемое устройство:", device)

# Загружаем предобученную модель ResNet-50
model_ft = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Заменяем последний слой, чтобы он соответствовал нашему количеству классов
num_ftrs = model_ft.fc.in_features
# Добавим Dropout перед последним слоем как легкую регуляризацию
model_ft.fc = nn.Sequential(
    nn.Dropout(p=0.2),
    nn.Linear(num_ftrs, len(class_names))
)

# Перемещаем модель на GPU, если он доступен
model_ft = model_ft.to(device)

# Определяем функцию потерь (loss function) и оптимизатор
# Если сильный дисбаланс — используем взвешенную функцию потерь
if imbalance_ratio >= 1.5:
    weights_list = [1.0 / max(1, class_counts.get(i, 1)) for i in range(len(class_names))]
    class_weights = torch.tensor(weights_list, dtype=torch.float32, device=device)
    criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.05)
    print(f"Используем взвешенную CrossEntropyLoss с весами: {weights_list}")
else:
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Дифференцированные скорости обучения: голова — быстрее, спина — медленнее
backbone_params = []
head_params = []
for name, p in model_ft.named_parameters():
    if name.startswith('fc.'):
        head_params.append(p)
    else:
        backbone_params.append(p)

optimizer_ft = optim.AdamW([
    {"params": backbone_params, "lr": 1e-4},
    {"params": head_params, "lr": 1e-3},
], weight_decay=1e-4)

# План понижения lr (плавное) — косинусная схема
# Подгоним период косинусного расписания под планируемое число эпох
scheduler = CosineAnnealingLR(optimizer_ft, T_max=20)

def train_model(
    model: nn.Module,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
    scheduler: torch.optim.lr_scheduler._LRScheduler,
    num_epochs: int = 15,
    patience: int = 5,
    use_amp: bool = True,
    freeze_epochs: int = 2,
) -> Tuple[nn.Module, Dict[str, float]]:
    """
    Функция для обучения модели с ранней остановкой и сохранением лучшей версии по валидации.
    Возвращает лучшую модель и словарь метрик.
    """
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    best_acc = 0.0
    best_loss = math.inf
    best_wts = copy.deepcopy(model.state_dict())
    epochs_no_improve = 0

    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

    # Фаза с заморозкой бэкбона (кроме головы) — стабилизация обучения на малых датасетах
    def set_backbone_trainable(is_trainable: bool):
        for name, p in model.named_parameters():
            if not name.startswith('fc.'):
                p.requires_grad = is_trainable

    if freeze_epochs > 0:
        set_backbone_trainable(False)
        print(f"Backbone заморожен на первые {freeze_epochs} эпох(и).")

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                optimizer.zero_grad(set_to_none=True)

                with torch.set_grad_enabled(phase == 'train'):
                    with torch.cuda.amp.autocast(enabled=use_amp):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                    if phase == 'train':
                        scaler.scale(loss).backward()
                        # Опционально: клиппинг градиентов для стабильности
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                        scaler.step(optimizer)
                        scaler.update()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            history[f"{phase}_loss"].append(epoch_loss)
            history[f"{phase}_acc"].append(epoch_acc.item())

            if phase == 'val':
                # шаг планировщика после валидации за эпоху
                scheduler.step()

                improved = epoch_acc.item() > best_acc
                if improved:
                    best_acc = epoch_acc.item()
                    best_loss = epoch_loss
                    best_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0
                    torch.save(best_wts, 'resnet_car_classifier_best.pth')
                    print('✓ Сохранена новая лучшая модель (по val Acc).')
                else:
                    epochs_no_improve += 1
                    if epochs_no_improve >= patience:
                        print(f'Ранняя остановка: без улучшений {epochs_no_improve} эпох подряд.')
                        model.load_state_dict(best_wts)
                        return model, {"best_val_acc": best_acc, "best_val_loss": best_loss}

        # По окончании эпохи размораживаем бэкбон, если подошло время
        if freeze_epochs > 0 and epoch + 1 == freeze_epochs:
            set_backbone_trainable(True)
            print("Backbone разморожен для дальнейшего дообучения.")

        # Печатаем текущие LR для наглядности
        lrs = [group['lr'] for group in optimizer.param_groups]
        print('LRs:', [f"{lr:.2e}" for lr in lrs])

    model.load_state_dict(best_wts)
    return model, {"best_val_acc": best_acc, "best_val_loss": best_loss}

# Запускаем обучение
model_trained, metrics = train_model(
    model_ft, criterion, optimizer_ft, scheduler,
    num_epochs=20, patience=5, use_amp=torch.cuda.is_available(), freeze_epochs=2,
)

# Сохраняем финальную (лучшую) модель
torch.save(model_trained.state_dict(), 'resnet_car_classifier.pth')
print("Модель успешно сохранена! Лучшая по валидации также сохранена как 'resnet_car_classifier_best.pth'.")

In [ ]:
# Загружаем модель
best_model_path = 'resnet_car_classifier_best.pth'
model_trained = model_ft
map_location = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_trained.load_state_dict(torch.load(best_model_path, map_location=map_location))
model_trained.eval()

# Проверяем количество классов в модели и в датасете
num_model_classes = model_trained.fc[-1].out_features
num_dataloader_classes = len(val_dataloader.dataset.classes)

if num_model_classes != num_dataloader_classes:
    raise ValueError(f"Несоответствие количества классов! Модель обучена на {num_model_classes} классах, а в датасете {num_dataloader_classes} классов.")

# Теперь запускаем визуализацию
visualize_results(model_trained, val_dataloader, class_names, map_location)